# Scrittopoli Quarti di finale

In [1]:
%matplotlib inline

# Import the necessary packages and modules
import matplotlib
matplotlib.style.use('ggplot')
import matplotlib.pyplot as plt
import matplotlib.dates  as mpdt
import matplotlib.ticker as mptk
import matplotlib.mlab as mlab

import numpy as np
import pytz
mytz=pytz.timezone("Europe/Rome")
import pandas as pd
import datetime as dt
import math
import importlib
import random
import itertools
import csv
import string

import mylib.spreadsheet as ms
import mylib.elo as melo
import mylib.utility as mu


In [2]:
# Generale

day="20171214_142354" # definitivo
calendario_csv="data/calendario_%s.csv" % day
gironi_csv="data/gironi_%s.csv" % day
tabellone_csv="data/tabellone_%s.csv" % day
ranking_csv="data/ranking_%s.csv" % day
scrittopoli_csv="data/scrittopoli_%s.csv" % day
elo_giocatori_csv="data/elo_giocatori_%s.csv" % day
elo_squadre_csv="data/elo_squadre_%s.csv" % day


calendario=pd.read_csv(calendario_csv,sep=":",quotechar='"',header=0,index_col=1).sort_index()
calendario=calendario.set_index(["girone","partita"], append=True)

gironi=pd.read_csv(gironi_csv,sep=":",quotechar='"',header=0)

tabellone=pd.read_csv(tabellone_csv,sep=",",quotechar='"',header=0,index_col=0)
elo_giocatori=pd.read_csv(elo_giocatori_csv,sep=",",quotechar='"',header=0,index_col=0)

ranking=pd.read_csv(ranking_csv,sep=",",quotechar='"',header=0)
ranking=ranking[ranking["squadra"]!="(riposo)"]
ranking=ranking.set_index("squadra")

scrittopoli=pd.read_csv(scrittopoli_csv,sep=",",quotechar='"',header=0)
scrittopoli=scrittopoli.set_index(["squadra","giocatore"])

elo_squadre=pd.read_csv(elo_squadre_csv,sep=",",quotechar='"',header=0)
elo_squadre=elo_squadre.set_index(["squadra","giocatore"])



# Giornata 1

label="giornata_1"
schedina_csv="data/schedina_"+label+".csv"
classifica_csv="data/classifica_"+label+".csv"
elo_g_csv="data/elo_g_"+label+".csv"
elo_s_csv="data/elo_s_"+label+".csv"

schedina_g1=pd.read_csv(schedina_csv,sep=",",quotechar='"')
classifica_g1=pd.read_csv(classifica_csv,sep=",",quotechar='"').set_index(["girone","squadra"])
elo_g_g1=pd.read_csv(elo_g_csv,sep=",",quotechar='"').set_index("giocatore")
elo_s_g1=pd.read_csv(elo_s_csv,sep=",",quotechar='"').set_index("squadra")

# Giornata 2

label="giornata_2"
schedina_csv="data/schedina_"+label+".csv"
classifica_csv="data/classifica_"+label+".csv"
elo_g_csv="data/elo_g_"+label+".csv"
elo_s_csv="data/elo_s_"+label+".csv"

schedina_g2=pd.read_csv(schedina_csv,sep=",",quotechar='"')
classifica_g2=pd.read_csv(classifica_csv,sep=",",quotechar='"').set_index(["girone","squadra"])
elo_g_g2=pd.read_csv(elo_g_csv,sep=",",quotechar='"').set_index("giocatore")
elo_s_g2=pd.read_csv(elo_s_csv,sep=",",quotechar='"').set_index("squadra")

# Giornata 3

label="giornata_3"
schedina_csv="data/schedina_"+label+".csv"
classifica_csv="data/classifica_"+label+".csv"
elo_g_csv="data/elo_g_"+label+".csv"
elo_s_csv="data/elo_s_"+label+".csv"
quarti_csv="data/quarti_"+label+".csv"

schedina_g3=pd.read_csv(schedina_csv,sep=",",quotechar='"')
classifica_g3=pd.read_csv(classifica_csv,sep=",",quotechar='"').set_index(["girone","squadra"])
elo_g_g3=pd.read_csv(elo_g_csv,sep=",",quotechar='"').set_index("giocatore")
elo_s_g3=pd.read_csv(elo_s_csv,sep=",",quotechar='"').set_index("squadra")
quarti=pd.read_csv(quarti_csv,sep=",",quotechar='"')

# Squadre

squadre_csv="data/squadre3.csv" 
squadre=pd.read_csv(squadre_csv,sep=":",quotechar='"',header=0)
squadre=squadre.set_index(["squadra"])


GIORNATA=4


## Come si svolgono gli incontri

Ogni capitano presenta la formazione (due titolari e una riserva). I due titolari vengono accoppiati agli avversari sulla base del punteggio Elo (i due più alti tra loro e i due più bassi tra loro).

Inseriscono i due racconti. Cose da controllare:
* link al racconto;
* tag;
* lunghezza del racconto;
* link al commento;
* commento valido.

Per la votazione ogni squadra presenta l'elenco dei voti da dare a quelle degli altri gironi. Per ogni partita danno quattro punteggi. Oltre a queste ci sono le penalità dell'arbitro.


In [3]:
quarti

,partita,squadra 1,squadra 2,vs
0,Q1,D Euforiche,Gli scrittori del quartierino,contro
1,Q2,I Bananari,Le TreMende,contro
2,Q3,Green House,Living Stones,contro
3,Q4,Armata Brancaleone,Gli ultimi,contro


In [4]:
mu.to_table_html(quarti.set_index("partita")[["squadra 1","vs","squadra 2"]],with_index=True)

<table style='margin:auto;' border='1' width='90%'>
<tr><th>&nbsp;</th>
<th>squadra 1</th>
<th>vs</th>
<th>squadra 2</th>
</tr>
<tr>
<th>Q1</th>
<td style='text-align:center;'>D Euforiche</td>
<td style='text-align:center;'>contro</td>
<td style='text-align:center;'>Gli scrittori del quartierino</td>
</tr>
<tr>
<th>Q2</th>
<td style='text-align:center;'>I Bananari</td>
<td style='text-align:center;'>contro</td>
<td style='text-align:center;'>Le TreMende</td>
</tr>
<tr>
<th>Q3</th>
<td style='text-align:center;'>Green House</td>
<td style='text-align:center;'>contro</td>
<td style='text-align:center;'>Living Stones</td>
</tr>
<tr>
<th>Q4</th>
<td style='text-align:center;'>Armata Brancaleone</td>
<td style='text-align:center;'>contro</td>
<td style='text-align:center;'>Gli ultimi</td>
</tr>
</table>


## Fase 1. Dalle formazioni ai match

In [5]:
#squadre=scrittopoli.copy()
#squadre=squadre.reset_index()[["squadra","giocatore"]].set_index("squadra")

squadre_in_gara=quarti[["partita","squadra 1"]]
squadre_in_gara.columns=["partita","squadra 2"]
squadre_in_gara=squadre_in_gara.append(quarti[["partita","squadra 2"]])
squadre_in_gara.columns=["partita","squadra"]
squadre_in_gara=squadre_in_gara.set_index("partita").sort_index()

importlib.reload(ms)
s=ms.N2017Spreadsheet()

old_data=s.get_formazioni(GIORNATA)
s.set_formazioni(GIORNATA,[],squadre_in_gara["squadra"],old_data)



In [6]:
# Versione random di prova, da sostituire con lettura della formazione via spreadsheet sotto
#data=[]
#for sq in squadre.index.unique():
#    capitano=squadre.loc[sq].sample(n=1).iloc[0]["giocatore"]
#    riserva=squadre.loc[sq].sample(n=1).iloc[0]["giocatore"]
#    titolari=list(squadre[squadre["giocatore"]!=riserva].loc[sq]["giocatore"])
#    data.append([sq,capitano,riserva]+titolari)
#formazioni=pd.DataFrame(data,columns=["squadra","capitano","riserva","titolare 1","titolare 2"])
#formazioni=formazioni.set_index("squadra")
#formazioni["riposa"]=""
#formazioni.set_value(riposano.iloc[0]["squadra 1"],"riposa","(riposa)")

formazioni=s.get_formazioni(GIORNATA)


In [7]:
formazioni

,riposa,capitano,riserva,titolare 1,titolare 2
squadra,,,,,
D Euforiche,,Kuno,Marcello,queffe,Kuno
Gli scrittori del quartierino,,Marty12,Federico72,camparino,Marty12
I Bananari,,Niko,massimopud,Niko,simone volponi
Le TreMende,,Emy,Thea,Emy,sefora
Green House,,Bango Skank,Bango Skank,Joyopi,Plata
Living Stones,,Macleo,Macleo,Vincenzo Iennaco,Ljuset
Armata Brancaleone,,AdStr,Skorpiòs,Ella F.,AdStr
Gli ultimi,,crazycat,crazycat,caipiroska,Ghigo


In [8]:
a=formazioni[["riserva"]].copy()
b=formazioni[["titolare 1"]].copy()
c=formazioni[["titolare 2"]].copy()
a.columns=["giocatore"]
b.columns=["giocatore"]
c.columns=["giocatore"]
giocatori=a.append(b).append(c)
giocatori=giocatori[giocatori["giocatore"]!=""]
giocatori=giocatori.reset_index().set_index("giocatore")
giocatori

,squadra
giocatore,
Marcello,D Euforiche
Federico72,Gli scrittori del quartierino
massimopud,I Bananari
Thea,Le TreMende
Bango Skank,Green House
Macleo,Living Stones
Skorpiòs,Armata Brancaleone
crazycat,Gli ultimi
queffe,D Euforiche


In [9]:
importlib.reload(melo)

elo_prev=elo_g_g3.copy()
elo_prev.columns=["squadra","rank %d" % (GIORNATA-1)]

elo_prev.merge(giocatori,left_index=True,right_index=True,how="outer")

giocatori=giocatori.reset_index().set_index(["squadra","giocatore"])
elo_prev=elo_prev.reset_index().set_index(["squadra","giocatore"])
elo_prev=elo_prev.merge(giocatori,left_index=True,right_index=True,how="outer")
elo_prev=elo_prev.fillna(1500)
elo_prev=elo_prev.reset_index().set_index("giocatore")

# c'è una componente casuale, una volta pubblicati vanno fissati
accoppiamenti=melo.calcola_accoppiamenti_finali(GIORNATA,formazioni,
                                                quarti.set_index("partita"),elo_prev)
accoppiamenti.reset_index().as_matrix()

data=[['Q1', 'D Euforiche', 'Kuno', 'Marcello', 'Kuno', 'queffe'],
       ['Q1', 'Gli scrittori del quartierino', 'Marty12', 'Federico72',
        'camparino', 'Marty12'],
       ['Q2', 'I Bananari', 'Niko', 'massimopud', 'Niko', 'simone volponi'],
       ['Q2', 'Le TreMende', 'Emy', 'Thea', 'sefora', 'Emy'],
       ['Q3', 'Green House', 'Bango Skank', 'Bango Skank', 'Plata',
        'Joyopi'],
       ['Q3', 'Living Stones', 'Macleo', 'Macleo', 'Ljuset',
        'Vincenzo Iennaco'],
       ['Q4', 'Armata Brancaleone', 'AdStr', 'Skorpiòs', 'Ella F.', 'AdStr'],
       ['Q4', 'Gli ultimi', 'crazycat', 'crazycat', 'Ghigo', 'caipiroska']]

accoppiamenti=pd.DataFrame(data,columns=["partita","squadra","capitano",
                                         "riserva","match 1",
                                         "match 2"]).set_index(["partita"])


accoppiamenti



,squadra,capitano,riserva,match 1,match 2
partita,,,,,
Q1,D Euforiche,Kuno,Marcello,Kuno,queffe
Q1,Gli scrittori del quartierino,Marty12,Federico72,camparino,Marty12
Q2,I Bananari,Niko,massimopud,Niko,simone volponi
Q2,Le TreMende,Emy,Thea,sefora,Emy
Q3,Green House,Bango Skank,Bango Skank,Plata,Joyopi
Q3,Living Stones,Macleo,Macleo,Ljuset,Vincenzo Iennaco
Q4,Armata Brancaleone,AdStr,Skorpiòs,Ella F.,AdStr
Q4,Gli ultimi,crazycat,crazycat,Ghigo,caipiroska


In [10]:

for partita in accoppiamenti.index.unique():
    p1=accoppiamenti.loc[partita].iloc[0]
    p2=accoppiamenti.loc[partita].iloc[1]
    print("<p>%s. %s - %s</p>" % (partita,p1["squadra"],p2["squadra"]) )
    print("<p style='margin-left: 40px;'>match 1: %s - %s</p>" % (p1["match 1"],p2["match 1"]))
    print("<p style='margin-left: 40px;'>match 2: %s - %s</p>" % (p1["match 2"],p2["match 2"]))
    

<p>Q1. D Euforiche - Gli scrittori del quartierino</p>
<p style='margin-left: 40px;'>match 1: Kuno - camparino</p>
<p style='margin-left: 40px;'>match 2: queffe - Marty12</p>
<p>Q2. I Bananari - Le TreMende</p>
<p style='margin-left: 40px;'>match 1: Niko - sefora</p>
<p style='margin-left: 40px;'>match 2: simone volponi - Emy</p>
<p>Q3. Green House - Living Stones</p>
<p style='margin-left: 40px;'>match 1: Plata - Ljuset</p>
<p style='margin-left: 40px;'>match 2: Joyopi - Vincenzo Iennaco</p>
<p>Q4. Armata Brancaleone - Gli ultimi</p>
<p style='margin-left: 40px;'>match 1: Ella F. - Ghigo</p>
<p style='margin-left: 40px;'>match 2: AdStr - caipiroska</p>


## Fase 2. Creazione del tabellone per voti e controlli

In [11]:
importlib.reload(ms)
s=ms.N2017Spreadsheet()

altre_squadre=squadre.loc[[ x not in list(squadre_in_gara["squadra"]) for x in squadre.index]].index.unique()

# togliere quando c'è la versione definitiva delle formazioni
#old_data=None
old_data=s.get_giornata_finali(GIORNATA)
s.set_giornata_finali(GIORNATA,accoppiamenti,squadre_in_gara,altre_squadre,old_data)


### Tabella ammissioni

In [12]:
importlib.reload(ms)

s=ms.N2017Spreadsheet()
ammissioni=s.get_giornata_finali(GIORNATA)
ammissioni=ammissioni[["base","verifiche"]]

old_cols=ammissioni.columns.tolist()
new_cols=[c[1] for c in old_cols]
new_cols

ammissioni.columns=new_cols
a=accoppiamenti.reset_index().set_index("match 1")[["partita","squadra","capitano","riserva"]]
a["match"]=1
a
b=accoppiamenti.reset_index().set_index("match 2")[["partita","squadra","capitano","riserva"]]
b["match"]=2

ammissioni=ammissioni.merge(a.append(b),left_index=True,right_index=True)
new_index=["partita","match","squadra"]
new_columns=["index","riserva","ingresso riserva",
             "titolo","lunghezza","penalità","tag","link","link commento","commento valido"]

ammissioni=ammissioni.reset_index().set_index(new_index).sort_index()[new_columns]
new_columns=["titolare","riserva","ingresso riserva",
             "titolo","lunghezza","penalità","tag","link","link commento","commento valido"]
ammissioni.columns=new_columns

def calc_ammissioni(row):
    row["sostituzione"]=""
    if row["ingresso riserva"]:
        sost="("+row["riserva"]+" da "+row["ingresso riserva"]+")"
        row["sostituzione"]=sost
    row["punti penalità"]=""
    if int(row["penalità"])==0: return row
    if abs(int(row["penalità"]))==1:
        row["punti penalità"]="1 punto di penalità"
    else:
        row["punti penalità"]="%d punti di penalità" % abs(int(row["penalità"]))
    return row

ammissioni=ammissioni.apply(calc_ammissioni,axis=1)

def str_penalita(p):
    p=int(p)
    if p==0: return ""
    return "%d goal" % p
    

ammissioni["giocatore"]=ammissioni["titolare"]+" "+ammissioni["sostituzione"]
ammissioni["str penalità"]=ammissioni["penalità"].apply(str_penalita)

ammissioni[["giocatore","titolo","lunghezza","str penalità"]]



giocatore  \
partita match squadra                                            
Q1      1     D Euforiche                                Kuno    
              Gli scrittori del quartierino         camparino    
        2     D Euforiche                              queffe    
              Gli scrittori del quartierino           Marty12    
Q2      1     I Bananari                                 Niko    
              Le TreMende                              sefora    
        2     I Bananari                       simone volponi    
              Le TreMende                                 Emy    
Q3      1     Green House                               Plata    
              Living Stones                            Ljuset    
        2     Green House                              Joyopi    
              Living Stones                  Vincenzo Iennaco    
Q4      1     Armata Brancaleone                      Ella F.    
              Gli ultimi                                Ghigo    
        2     Armata Brancaleone                        AdStr    
              Gli ultimi                           caipiroska    

                                                                                     titolo  \
partita match squadra                                                                         
Q1      1     D Euforiche                                    Il meglio che sa fare un pesce   
              Gli scrittori del quartierino                           Il negozio di acquari   
        2     D Euforiche                                             Il presente di Brando   
              Gli scrittori del quartierino     Pensavo fosse amore e invece era un dugongo   
Q2      1     I Bananari                                      Non c’è nessun vetro, tra noi   
              Le TreMende                                            Le sette dell’acquario   
        2     I Bananari                                                           WaterEnd   
              Le TreMende                                              Una festa a sorpresa   
Q3      1     Green House                                                  L'acquario vuoto   
              Living Stones                                                         Elenoir   
        2     Green House                                                      Quanta magia   
              Living Stones                                      Looking through the window   
Q4      1     Armata Brancaleone                                       Pesci in un acquario   
              Gli ultimi                     Quel giorno in cui mio padre comprò l’acquario   
        2     Armata Brancaleone                                                 Sgradevole   
              Gli ultimi                                                     Cuore di mamma   

                                            lunghezza str penalità  
partita match squadra                                               
Q1      1     D Euforiche                        7733               
              Gli scrittori del quartierino      3216               
        2     D Euforiche                        6695               
              Gli scrittori del quartierino      6895               
Q2      1     I Bananari                         7999               
              Le TreMende                        7694               
        2     I Bananari                         7995               
              Le TreMende                        7597               
Q3      1     Green House                        6495               
              Living Stones                      7995               
        2     Green House                        6596               
              Living Stones                      5881               
Q4      1     Armata Brancaleone                 4304               
              Gli ultimi                         7902               
        2     Armata Brancaleone                 

In [13]:
mu.to_table_html_multi(ammissioni[["giocatore","titolo",
                                   "lunghezza"]])

<table style='margin:auto;' border='1' width='90%'>
<th>partita</th>
<th>match</th>
<th>squadra</th>
<th>giocatore</th>
<th>titolo</th>
<th>lunghezza</th>
</tr>
<tr>
<th rowspan='4'>Q1</th>
<th rowspan='2'>1</th>
<th rowspan='1'>D Euforiche</th>
<td style='text-align:center;'>Kuno </td>
<td style='text-align:center;'>Il meglio che sa fare un pesce</td>
<td style='text-align:center;'>7733</td>
</tr>
<tr>
<th rowspan='1'>Gli scrittori del quartierino</th>
<td style='text-align:center;'>camparino </td>
<td style='text-align:center;'>Il negozio di acquari</td>
<td style='text-align:center;'>3216</td>
</tr>
<tr>
<th rowspan='2'>2</th>
<th rowspan='1'>D Euforiche</th>
<td style='text-align:center;'>queffe </td>
<td style='text-align:center;'>Il presente di Brando</td>
<td style='text-align:center;'>6695</td>
</tr>
<tr>
<th rowspan='1'>Gli scrittori del quartierino</th>
<td style='text-align:center;'>Marty12 </td>
<td style='text-align:center;'>Pensavo fosse amore e invece era un dugongo</td>

## Fase 3. Recupero dati

In [14]:
importlib.reload(ms)

s=ms.N2017Spreadsheet()
voti=s.get_giornata_finali(GIORNATA)


In [15]:
### Simulazione di riempimento del tabellone - da togliere quando ci sono i voti definitivi
importlib.reload(mu)

SIMULA=False
SIMULA_AMMISSIONI=False
if SIMULA:
    if SIMULA_AMMISSIONI:
        # solo prima delle ammissioni definitive
        voti["base","titolo"]=voti["base","titolo"].apply(mu.random_title)
        voti["base","ingresso riserva"]=voti["base","ingresso riserva"].apply(mu.random_riserva)
        voti["verifiche","lunghezza"]=voti["verifiche","lunghezza"].apply(mu.random_len)
        voti["verifiche","link"]=voti["verifiche","link"].apply(mu.random_verifica)
        voti["verifiche","link commento"]=voti["verifiche","link commento"].apply(mu.random_verifica)
        voti["verifiche","tag"]=voti["verifiche","tag"].apply(mu.random_verifica)
        voti["verifiche","commento valido"]=voti["verifiche","commento valido"].apply(mu.random_verifica)
        voti["verifiche","penalità"]=voti["verifiche","lunghezza"].apply(mu.calc_penalita_lungh)
        for k in ["tag","link","link commento","commento valido"]:
            voti["verifiche","penalità"]+=voti["verifiche",k].apply(mu.calc_penalita)

    #for G,Gpart_list,altri in [ ("A",[1],[("B",1),("B",2),("C",1),("C",2)]),
    #                            ("B",[1,2],[("A",1),("C",1),("C",2)]),
    #                            ("C",[1,2],[("A",1),("B",1),("B",2)]) ]:
    
    for P,Pvotanti in [ ("Q1",["Q2","Q3","Q4"]),
                        ("Q2",["Q1","Q3","Q4"]),
                        ("Q3",["Q1","Q2","Q4"]),
                        ("Q4",["Q1","Q2","Q3"]) ]:
        for m in ["match 1","match 2"]:
            giocatore_1=accoppiamenti.loc[P][m].iloc[0]
            giocatore_2=accoppiamenti.loc[P][m].iloc[1]
            #for sq in squadre_in_gara.loc[P]["squadra"]:
            #    goal_1=0
            #    goal_2=0
            #    voti.set_value(giocatore_1,(P,sq),goal_1)
            #    voti.set_value(giocatore_2,(P,sq),goal_2)
            
            for part in Pvotanti:
                for sq in squadre_in_gara.loc[part]["squadra"]:
                    goal_1=random.randint(0,2)
                    goal_2=2-goal_1
                    voti.set_value(giocatore_1,(part,sq),goal_1)
                    voti.set_value(giocatore_2,(part,sq),goal_2)
            for sq in altre_squadre:
                goal_1=random.randint(0,2)
                goal_2=2-goal_1
                voti.set_value(giocatore_1,("non qualificate",sq),goal_1)
                voti.set_value(giocatore_2,("non qualificate",sq),goal_2)

voti

base  \
                 ingresso riserva   
titolare                            
Kuno                                
queffe                              
camparino                           
Marty12                             
Niko                                
simone volponi                      
sefora                              
Emy                                 
Plata                               
Joyopi                              
Ljuset                              
Vincenzo Iennaco                    
Ella F.                             
AdStr                               
Ghigo                               
caipiroska                          

                                                                 verifiche  \
                                                          titolo lunghezza   
titolare                                                                     
Kuno                              Il meglio che sa fare un pesce      7733   
queffe                                     Il presente di Brando      6695   
camparino                                  Il negozio di acquari      3216   
Marty12              Pensavo fosse amore e invece era un dugongo      6895   
Niko                               Non c’è nessun vetro, tra noi      7999   
simone volponi                                          WaterEnd      7995   
sefora                                    Le sette dell’acquario      7694   
Emy                                         Una festa a sorpresa      7597   
Plata                                           L'acquario vuoto      6495   
Joyopi                                              Quanta magia      6596   
Ljuset                                                   Elenoir      7995   
Vincenzo Iennaco                      Looking through the window      5881   
Ella F.                                     Pesci in un acquario      4304   
AdStr                                                 Sgradevole      6703   
Ghigo             Quel giorno in cui mio padre comprò l’acquario      7902   
caipiroska                                        Cuore di mamma      6173   

                                                                          Q1  \
                 tag link link commento commento valido penalità D Euforiche   
titolare                                                                       
Kuno               x    x             x               x        0               
queffe             x    x             x               x        0               
camparino          x    x             x               x        0               
Marty12            x    x             x               x        0               
Niko               x    x             x               x        0           1   
simone volponi     x    x             x               x        0           2   
sefora             x    x             x               x        0           1   
Emy                x    x             x               x        0           0   
Plata              x    x             x               x        0           2   
Joyopi             x    x             x               x        0           0   
Ljuset             x    x             x               x        0           0   
Vincenzo Iennaco   x    x             x               x        0           2   
Ella F.            x    x             x               x        0           0   
AdStr              x    x             x               x        0           1   
Ghigo              x    x             x               x        0           2   
caipiroska         x    x             x               x        0           1   

                                                       Q2              \
                 Gli scrittori del quartierino I Bananari Le TreMende   
titolare                                                                
Kuno                                                    0           1   
queffe                       

In [16]:
def insert_zeros(row):
    for part in squadre_in_gara.index.unique():
        for sq in squadre_in_gara.loc[part]["squadra"]:
            if row[(part,sq)]=="": row[(part,sq)]=0
    for sq in altre_squadre:
        if row[("non qualificate",sq)]=="": row[("non qualificate",sq)]=0
    return row

voti=voti.apply(insert_zeros,axis=1)

colonne_voto=list(squadre_in_gara.index.unique())
colonne_voto.append("non qualificate")

voti["verifiche","penalità"]=pd.to_numeric(voti["verifiche","penalità"],downcast="signed")
voti["base","goal giocatore"]=voti[colonne_voto].sum(axis=1,level=1).sum(axis=1)+voti["verifiche","penalità"]
voti["base","goal giocatore"]=pd.to_numeric(voti["base","goal giocatore"],downcast="signed")
voti

base  \
                 ingresso riserva   
titolare                            
Kuno                                
queffe                              
camparino                           
Marty12                             
Niko                                
simone volponi                      
sefora                              
Emy                                 
Plata                               
Joyopi                              
Ljuset                              
Vincenzo Iennaco                    
Ella F.                             
AdStr                               
Ghigo                               
caipiroska                          

                                                                 verifiche  \
                                                          titolo lunghezza   
titolare                                                                     
Kuno                              Il meglio che sa fare un pesce      7733   
queffe                                     Il presente di Brando      6695   
camparino                                  Il negozio di acquari      3216   
Marty12              Pensavo fosse amore e invece era un dugongo      6895   
Niko                               Non c’è nessun vetro, tra noi      7999   
simone volponi                                          WaterEnd      7995   
sefora                                    Le sette dell’acquario      7694   
Emy                                         Una festa a sorpresa      7597   
Plata                                           L'acquario vuoto      6495   
Joyopi                                              Quanta magia      6596   
Ljuset                                                   Elenoir      7995   
Vincenzo Iennaco                      Looking through the window      5881   
Ella F.                                     Pesci in un acquario      4304   
AdStr                                                 Sgradevole      6703   
Ghigo             Quel giorno in cui mio padre comprò l’acquario      7902   
caipiroska                                        Cuore di mamma      6173   

                                                                          Q1  \
                 tag link link commento commento valido penalità D Euforiche   
titolare                                                                       
Kuno               x    x             x               x        0           0   
queffe             x    x             x               x        0           0   
camparino          x    x             x               x        0           0   
Marty12            x    x             x               x        0           0   
Niko               x    x             x               x        0           1   
simone volponi     x    x             x               x        0           2   
sefora             x    x             x               x        0           1   
Emy                x    x             x               x        0           0   
Plata              x    x             x               x        0           2   
Joyopi             x    x             x               x        0           0   
Ljuset             x    x             x               x        0           0   
Vincenzo Iennaco   x    x             x               x        0           2   
Ella F.            x    x             x               x        0           0   
AdStr              x    x             x               x        0           1   
Ghigo              x    x             x               x        0           2   
caipiroska         x    x             x               x        0           1   

                                                       Q2              \
                 Gli scrittori del quartierino I Bananari Le TreMende   
titolare                                                                
Kuno                                         0          0           1   
queffe                       

## Calcolo dei risultati

### Risultati dei quarti di finale


In [17]:
risultati=accoppiamenti.copy()

risultati=risultati.reset_index().set_index("squadra")
risultati["match 1 goal"]=0
risultati["match 1 riserva"]=False
risultati["match 2 goal"]=0
risultati["match 2 riserva"]=False
risultati["goal"]=0
risultati["penalità"]=0


for sq in risultati.index:
    m1=risultati.loc[sq]["match 1"]
    m2=risultati.loc[sq]["match 2"]
    goal_1=voti.loc[m1]["base","goal giocatore"]
    goal_2=voti.loc[m2]["base","goal giocatore"]
    pen_1=voti.loc[m1]["verifiche","penalità"]
    pen_2=voti.loc[m2]["verifiche","penalità"]
    risultati.set_value(sq,"match 1 goal",goal_1)
    risultati.set_value(sq,"match 2 goal",goal_2)
    risultati.set_value(sq,"goal",goal_1+goal_2)
    risultati.set_value(sq,"penalità",pen_1+pen_2)
    ris_1=bool(voti.loc[m1]["base","ingresso riserva"])
    risultati.set_value(sq,"match 1 riserva",ris_1)
    if ris_1: continue
    risultati.set_value(sq,"match 2 riserva",bool(voti.loc[m2]["base","ingresso riserva"]))
    
risultati=risultati.reset_index().set_index(["partita"])
sc_dati=[]
for partita in risultati.index.unique():
    squadra_1=risultati.loc[partita]["squadra"].iloc[0]
    squadra_2=risultati.loc[partita]["squadra"].iloc[1]
    goal_1=risultati.loc[partita]["goal"].iloc[0]
    goal_2=risultati.loc[partita]["goal"].iloc[1]
    pen_1=risultati.loc[partita]["penalità"].iloc[0]
    pen_2=risultati.loc[partita]["penalità"].iloc[1]
    m1_goal_1=risultati.loc[partita]["match 1 goal"].iloc[0]
    m1_goal_2=risultati.loc[partita]["match 1 goal"].iloc[1]
    m2_goal_1=risultati.loc[partita]["match 2 goal"].iloc[0]
    m2_goal_2=risultati.loc[partita]["match 2 goal"].iloc[1]
    sc_dati.append([partita,squadra_1,squadra_2,goal_1,goal_2,
                    m1_goal_1,m1_goal_2,m2_goal_1,m2_goal_2,pen_1,pen_2])

risultati=risultati.reset_index().set_index(["partita","squadra"])

schedina=pd.DataFrame(sc_dati,columns=["partita","squadra 1","squadra 2",
                                       "goal squadra 1","goal squadra 2",
                                       "match 1 squadra 1","match 1 squadra 2",
                                       "match 2 squadra 1","match 2 squadra 2",
                                       "penalità squadra 1","penalità squadra 2"])

out_schedina=schedina.copy()
out_schedina["titolo"]=out_schedina["squadra 1"]+" - "+out_schedina["squadra 2"]
out_schedina["risultato"]=( out_schedina["goal squadra 1"].apply(str)
                            +" - "+out_schedina["goal squadra 2"].apply(str) )
out_schedina["match 1"]=( out_schedina["match 1 squadra 1"].apply(str)
                          +" - "+out_schedina["match 1 squadra 2"].apply(str) )
out_schedina["match 2"]=( out_schedina["match 2 squadra 1"].apply(str)
                          +" - "+out_schedina["match 2 squadra 2"].apply(str) )
out_schedina=out_schedina.set_index(["partita"])

out_schedina=out_schedina[["titolo","match 1","match 2","risultato"]]


risultati

capitano      riserva    match 1  \
partita squadra                                                              
Q1      D Euforiche                           Kuno     Marcello       Kuno   
        Gli scrittori del quartierino      Marty12   Federico72  camparino   
Q2      I Bananari                            Niko   massimopud       Niko   
        Le TreMende                            Emy         Thea     sefora   
Q3      Green House                    Bango Skank  Bango Skank      Plata   
        Living Stones                       Macleo       Macleo     Ljuset   
Q4      Armata Brancaleone                   AdStr     Skorpiòs    Ella F.   
        Gli ultimi                        crazycat     crazycat      Ghigo   

                                                match 2  match 1 goal  \
partita squadra                                                         
Q1      D Euforiche                              queffe             5   
        Gli scrittori del quartierino           Marty12             9   
Q2      I Bananari                       simone volponi             8   
        Le TreMende                                 Emy             6   
Q3      Green House                              Joyopi            10   
        Living Stones                  Vincenzo Iennaco             4   
Q4      Armata Brancaleone                        AdStr             0   
        Gli ultimi                           caipiroska            14   

                                       match 1 riserva  match 2 goal  \
partita squadra                                                        
Q1      D Euforiche                              False            13   
        Gli scrittori del quartierino            False             1   
Q2      I Bananari                               False            14   
        Le TreMende                              False             0   
Q3      Green House                              False             7   
        Living Stones                            False             7   
Q4      Armata Brancaleone                       False             9   
        Gli ultimi                               False             5   

                                       match 2 riserva  goal  penalità  
partita squadra                                                         
Q1      D Euforiche                              False    18         0  
        Gli scrittori del quartierino            False    10         0  
Q2      I Bananari                               False    22         0  
        Le TreMende                              False     6         0  
Q3      Green House                              False    17         0  
        Living Stones                            False    11         0  
Q4      Armata Brancaleone                       False     9         0  
        Gli ultimi                               False    19         0

In [18]:
schedina

,partita,squadra 1,squadra 2,goal squadra 1,goal squadra 2,match 1 squadra 1,match 1 squadra 2,match 2 squadra 1,match 2 squadra 2,penalità squadra 1,penalità squadra 2
0,Q1,D Euforiche,Gli scrittori del quartierino,18,10,5,9,13,1,0,0
1,Q2,I Bananari,Le TreMende,22,6,8,6,14,0,0,0
2,Q3,Green House,Living Stones,17,11,10,4,7,7,0,0
3,Q4,Armata Brancaleone,Gli ultimi,9,19,0,14,9,5,0,0


In [19]:
out_schedina

,titolo,match 1,match 2,risultato
partita,,,,
Q1,D Euforiche - Gli scrittori del quartierino,5 - 9,13 - 1,18 - 10
Q2,I Bananari - Le TreMende,8 - 6,14 - 0,22 - 6
Q3,Green House - Living Stones,10 - 4,7 - 7,17 - 11
Q4,Armata Brancaleone - Gli ultimi,0 - 14,9 - 5,9 - 19


In [20]:
# output per il post dei risultati

mu.to_table_html(out_schedina,with_index=True)

<table style='margin:auto;' border='1' width='90%'>
<tr><th>&nbsp;</th>
<th>titolo</th>
<th>match 1</th>
<th>match 2</th>
<th>risultato</th>
</tr>
<tr>
<th>Q1</th>
<td style='text-align:center;'>D Euforiche - Gli scrittori del quartierino</td>
<td style='text-align:center;'>5 - 9</td>
<td style='text-align:center;'>13 - 1</td>
<td style='text-align:center;'>18 - 10</td>
</tr>
<tr>
<th>Q2</th>
<td style='text-align:center;'>I Bananari - Le TreMende</td>
<td style='text-align:center;'>8 - 6</td>
<td style='text-align:center;'>14 - 0</td>
<td style='text-align:center;'>22 - 6</td>
</tr>
<tr>
<th>Q3</th>
<td style='text-align:center;'>Green House - Living Stones</td>
<td style='text-align:center;'>10 - 4</td>
<td style='text-align:center;'>7 - 7</td>
<td style='text-align:center;'>17 - 11</td>
</tr>
<tr>
<th>Q4</th>
<td style='text-align:center;'>Armata Brancaleone - Gli ultimi</td>
<td style='text-align:center;'>0 - 14</td>
<td style='text-align:center;'>9 - 5</td>
<td style='text-align:c

### Passaggio del turno

In [21]:
# da modificare in base ai pareggi
SCELTE_ARBITRO={
    "Q1": {"vince": "D Euforiche", "perde": "Gli scrittori del quartierino"},
    "Q2": {"vince": "Le TreMende","perde": "I Bananari" },
    "Q3": {"vince": "Green House","perde": "Living Stones" },
    "Q4": {"vince": "Armata Brancaleone","perde": "Gli ultimi" },   
}

q=schedina.copy()

def vinceperde(row):
    if row["goal squadra 1"]>row["goal squadra 2"]:
        row["vince"]=row["squadra 1"]
        row["perde"]=row["squadra 2"]
    elif row["goal squadra 1"]<row["goal squadra 2"]:
        row["vince"]=row["squadra 2"]
        row["perde"]=row["squadra 1"]
    else:
        p=row["partita"]
        row["vince"]=SCELTE_ARBITRO[p]["vince"]
        row["perde"]=SCELTE_ARBITRO[p]["perde"]
    return row
        
q=q.apply(vinceperde,axis=1).set_index("partita")

data=[ ["S1",q.get_value("Q1","vince"),q.get_value("Q2","vince")],
       ["S2",q.get_value("Q3","vince"),q.get_value("Q4","vince")] ]

semifinali=pd.DataFrame(data,columns=["partita","squadra 1","squadra 2"])
semifinali["vs"]="contro"
semifinali

,partita,squadra 1,squadra 2,vs
0,S1,D Euforiche,I Bananari,contro
1,S2,Green House,Gli ultimi,contro


## Nuovi punteggi Elo

In [22]:
def calc_giocatore(row,match):
    riserva=row["riserva"]
    giocatore=row["match %d" % match]
    sostituzione=row["match %d riserva" % match]
    if sostituzione: return riserva
    return giocatore

def calc_giocatore_1(row): return calc_giocatore(row,1)
def calc_giocatore_2(row): return calc_giocatore(row,2)

def calcola_punteggi(row):
    exp_a,exp_b=melo.punteggi_attesi(row["elo precedente"],row["elo avversario"])
    score_a,score_b=melo.punteggi_reali(row["goal fatti"],row["goal subiti"])
    row["punteggio atteso"]=exp_a
    row["punteggio reale"]=score_a
    return row

elo_g_prec=elo_g_g3
elo_s_prec=elo_g_prec.reset_index().set_index("squadra").sort_index().groupby(level=0).mean().copy()

a=schedina.copy()[["partita","squadra 1","squadra 2","goal squadra 1","goal squadra 2"]]
b=schedina.copy()[["partita","squadra 2","squadra 1","goal squadra 2","goal squadra 1"]]
a.columns=["partita","squadra","avversario","goal fatti","goal subiti"]
b.columns=["partita","squadra","avversario","goal fatti","goal subiti"]

elo_s=a.append(b).set_index("squadra")

elo_s_prec.columns=["elo precedente"]
elo_s=elo_s.merge(elo_s_prec,left_index=True,right_index=True)
elo_s_prec.columns=["elo avversario"]
elo_s=elo_s.reset_index().set_index("avversario").merge(elo_s_prec,left_index=True,
                                                        right_index=True)

elo_s=elo_s.reset_index().set_index("squadra")
elo_s.columns=["avversario","partita","goal fatti","goal subiti","elo precedente","elo avversario"]
elo_s=elo_s.apply(calcola_punteggi,axis=1)
elo_s["scostamento squadra"]=elo_s["punteggio reale"]-elo_s["punteggio atteso"]

#rip_sq=riposano.iloc[0]["squadra 1"]
#rip_mean=elo_g_prec[elo_g_prec["squadra"]==rip_sq]["elo"].mean()

non_qual_filter=[ (x in list(altre_squadre)) for x in elo_g_prec["squadra"]]
non_qual_mean=elo_g_prec[non_qual_filter].groupby("squadra").mean()

non_qual_mean.columns=["elo precedente"]
non_qual_mean["avversario"]="-"
non_qual_mean["partita"]="non qualificate"
non_qual_mean["goal fatti"]=0
non_qual_mean["goal subiti"]=0
non_qual_mean["elo avversario"]=non_qual_mean["elo precedente"]
non_qual_mean["punteggio atteso"]=0.5
non_qual_mean["punteggio reale"]=0.5
non_qual_mean["scostamento squadra"]=0

elo_s=elo_s.append(non_qual_mean)

scost_squadre=elo_s.reset_index()[["partita","squadra","scostamento squadra"]]
scost_squadre=scost_squadre.set_index(["partita","squadra"])

sq_gioc=squadre_in_gara.reset_index().set_index("squadra").copy()
sq_gioc=sq_gioc.merge(squadre,left_index=True,
                      right_index=True,how="outer").fillna("non qualificate")
sq_gioc=sq_gioc.reset_index().set_index(["partita","squadra"])

scost_squadre=sq_gioc.merge(scost_squadre,left_index=True,right_index=True,how="outer").fillna(0)
scost_squadre=scost_squadre.reset_index().set_index(["squadra","giocatore"])

scost_squadre=elo_g_prec.reset_index().set_index(["squadra",
                                                  "giocatore"]).merge(scost_squadre,
                                                                      left_index=True,
                                                                      right_index=True,
                                                                      how="outer")
scost_squadre=scost_squadre.reset_index().set_index(["giocatore","squadra","partita"])

scost_squadre


,,,elo,scostamento squadra
giocatore,squadra,partita,,
AdStr,Armata Brancaleone,Q4,1477,-0.176293
Ella F.,Armata Brancaleone,Q4,1562,-0.176293
Skorpiòs,Armata Brancaleone,Q4,1475,-0.176293
Kuno,D Euforiche,Q1,1557,0.076571
Marcello,D Euforiche,Q1,1506,0.076571
queffe,D Euforiche,Q1,1515,0.076571
Federico72,Gli scrittori del quartierino,Q1,1488,-0.076571
Marty12,Gli scrittori del quartierino,Q1,1451,-0.076571
camparino,Gli scrittori del quartierino,Q1,1500,-0.076571


In [23]:
q=risultati.copy()
q["giocatore 1"]=q.apply(calc_giocatore_1,axis=1)
q["giocatore 2"]=q.apply(calc_giocatore_2,axis=1)

a=q[["giocatore 1","match 1 goal"]].copy()
a.columns=["giocatore","goal"]
a["match"]="1"

b=q[["giocatore 2","match 2 goal"]].copy()
b.columns=["giocatore","goal"]
b["match"]="2"

q=a.append(b).reset_index().set_index(["partita","match"])
q=q.merge(q,how="outer",left_index=True,right_index=True,suffixes=[" 1"," 2"])
q=q[q["giocatore 1"]!=q["giocatore 2"]]

elo=q[["squadra 1","giocatore 1","giocatore 2","goal 1","goal 2"]]
elo.columns=["squadra","giocatore","avversario","goal fatti","goal subiti"]
elo=elo.reset_index().set_index(["partita","squadra","giocatore"]).sort_index()
elo=elo[["avversario","goal fatti","goal subiti"]].copy()

base_elo=elo_g_prec[["elo"]].copy()
base_elo=base_elo.loc[elo.reset_index()["giocatore"]].fillna(1500)

base_elo.columns=["elo precedente"]
elo=elo.reset_index().set_index("giocatore").merge(base_elo,left_index=True,right_index=True)
base_elo.columns=["elo avversario"]
elo=elo.reset_index().set_index("avversario").merge(base_elo,left_index=True,right_index=True)
elo=elo.reset_index().set_index(["partita","squadra","giocatore"])
elo.columns=["avversario","goal fatti","goal subiti","elo precedente","elo avversario"]
elo=elo.apply(calcola_punteggi,axis=1)
elo["scostamento"]=elo["punteggio reale"]-elo["punteggio atteso"]

elo=elo.reset_index().set_index(["giocatore","squadra","partita"])
elo=elo.merge(scost_squadre[["scostamento squadra","elo"]],
              left_index=True,right_index=True,how="outer")

elo["elo precedente"]=elo["elo"]
elo["goal fatti"]=pd.to_numeric(elo["goal fatti"].fillna(0),downcast="signed")
elo["goal subiti"]=pd.to_numeric(elo["goal subiti"].fillna(0),downcast="signed")
elo["elo precedente"]=pd.to_numeric(elo["elo precedente"].fillna(1500),downcast="signed")
elo["elo avversario"]=pd.to_numeric(elo["elo avversario"].fillna(1500),downcast="signed")
elo["punteggio atteso"]=elo["punteggio atteso"].fillna(0.5)
elo["punteggio reale"]=elo["punteggio reale"].fillna(0.5)
elo["scostamento"]=elo["scostamento"].fillna(0.0)
elo["avversario"]=elo["avversario"].fillna("(non giocato)")

elo=elo.reset_index().set_index(["giocatore"])

L=list(elo.columns)
L.remove("elo")
elo=elo[L]

elo

,squadra,partita,avversario,goal fatti,goal subiti,elo precedente,elo avversario,punteggio atteso,punteggio reale,scostamento,scostamento squadra
giocatore,,,,,,,,,,,
AdStr,Armata Brancaleone,Q4,caipiroska,9,5,1477,1489,0.482737,0.642857,0.160120,-0.176293
AlexComan,Retroguardia Trash,non qualificate,(non giocato),0,0,1470,1500,0.500000,0.500000,0.000000,0.000000
Andrea28,Retroguardia Trash,non qualificate,(non giocato),0,0,1494,1500,0.500000,0.500000,0.000000,0.000000
AryaSophia,the 3 BrOthers,non qualificate,(non giocato),0,0,1450,1500,0.500000,0.500000,0.000000,0.000000
Bango Skank,Green House,Q3,(non giocato),0,0,1531,1500,0.500000,0.500000,0.000000,0.066935
Ella F.,Armata Brancaleone,Q4,Ghigo,0,14,1562,1500,0.588290,0.000000,-0.588290,-0.176293
Emy,Le TreMende,Q2,simone volponi,0,14,1470,1491,0.469815,0.000000,-0.469815,-0.268931
Eudes,Retroguardia Trash,non qualificate,(non giocato),0,0,1518,1500,0.500000,0.500000,0.000000,0.000000
Federico72,Gli scrittori del quartierino,Q1,(non giocato),0,0,1488,1500,0.500000,0.500000,0.000000,-0.076571


In [24]:
def calc_K(dfcol,max_variazione):
    mean=dfcol.mean()
    sigma=dfcol.std()
    for alpha in np.linspace(0,6,601):
        if mlab.normpdf(alpha*sigma,mean,sigma)<=0.0001:
            break
    max_scostamento=alpha*sigma
    K=max_variazione/max_scostamento
    return K

def norm_elo(x):
    e=round(max(1450,x))
    return int(e)

KG=calc_K(elo["scostamento"],70)
KS=calc_K(elo["scostamento squadra"],20)


elo["elo nuovo"]=elo["elo precedente"]+KG*elo["scostamento"]+KS*elo["scostamento squadra"]
elo["elo nuovo"]=elo["elo nuovo"].apply(norm_elo)

elo[["elo precedente","elo nuovo","scostamento","scostamento squadra"]].sort_values("elo nuovo",ascending=False)

,elo precedente,elo nuovo,scostamento,scostamento squadra
giocatore,,,,
Plata,1544,1560,0.189840,0.066935
Ghigo,1500,1548,0.588290,0.176293
Kuno,1557,1543,-0.224159,0.076571
queffe,1515,1542,0.337496,0.076571
ITG,1542,1542,0.000000,0.000000
Niko,1530,1540,0.019805,0.268931
crazycat,1530,1535,0.000000,0.176293
simone volponi,1491,1533,0.469815,0.268931
Bango Skank,1531,1533,0.000000,0.066935


In [25]:
elo_g=elo[["squadra","elo nuovo"]]
elo_g.columns=["squadra","elo"]
elo_g=elo_g.sort_values("elo",ascending=False)
elo_g


,squadra,elo
giocatore,,
Plata,Green House,1560
Ghigo,Gli ultimi,1548
Kuno,D Euforiche,1543
queffe,D Euforiche,1542
ITG,the 3 BrOthers,1542
Niko,I Bananari,1540
crazycat,Gli ultimi,1535
simone volponi,I Bananari,1533
Bango Skank,Green House,1533


In [26]:
mu.to_table_html(elo_g,with_index=True)

<table style='margin:auto;' border='1' width='90%'>
<tr><th>&nbsp;</th>
<th>squadra</th>
<th>elo</th>
</tr>
<tr>
<th>Plata</th>
<td style='text-align:center;'>Green House</td>
<td style='text-align:center;'>1560</td>
</tr>
<tr>
<th>Ghigo</th>
<td style='text-align:center;'>Gli ultimi</td>
<td style='text-align:center;'>1548</td>
</tr>
<tr>
<th>Kuno</th>
<td style='text-align:center;'>D Euforiche</td>
<td style='text-align:center;'>1543</td>
</tr>
<tr>
<th>queffe</th>
<td style='text-align:center;'>D Euforiche</td>
<td style='text-align:center;'>1542</td>
</tr>
<tr>
<th>ITG</th>
<td style='text-align:center;'>the 3 BrOthers</td>
<td style='text-align:center;'>1542</td>
</tr>
<tr>
<th>Niko</th>
<td style='text-align:center;'>I Bananari</td>
<td style='text-align:center;'>1540</td>
</tr>
<tr>
<th>crazycat</th>
<td style='text-align:center;'>Gli ultimi</td>
<td style='text-align:center;'>1535</td>
</tr>
<tr>
<th>simone volponi</th>
<td style='text-align:center;'>I Bananari</td>
<td style='

## Salvataggio dei risultati

In [27]:
## CSV
label="giornata_%d" % GIORNATA

# formazioni
formazioni_csv="data/formazioni_"+label+".csv"
formazioni.to_csv(formazioni_csv,index_label="squadra")

# accoppiamenti
accoppiamenti_csv="data/accoppiamenti_"+label+".csv"
accoppiamenti.to_csv(accoppiamenti_csv)

# voti
voti_csv="data/voti_"+label+".csv"
voti.to_csv(voti_csv)

# risultati
risultati_csv="data/risultati_"+label+".csv"
risultati.to_csv(risultati_csv)

# schedina
schedina_csv="data/schedina_"+label+".csv"
schedina.to_csv(schedina_csv,index=False)

# elo
elo_csv="data/elo_"+label+".csv"
elo.to_csv(elo_csv)

# elo_g
elo_g_csv="data/elo_g_"+label+".csv"
elo_g.to_csv(elo_g_csv)

# elo_g
elo_s_csv="data/elo_s_"+label+".csv"
elo_s.to_csv(elo_s_csv)

# ammissioni
ammissioni_csv="data/ammissioni_"+label+".csv"
ammissioni.to_csv(ammissioni_csv)

# semifinali
semifinali_csv="data/semifinali_"+label+".csv"
semifinali.to_csv(semifinali_csv,index=False)
